# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [37]:
import torch
import torch.nn as nn

In [38]:
print(torch.backends.mps.is_available())

True


In [39]:
if False and torch.backends.mps.is_available():
    device = torch.device("mps") # Apple GPU
    print("Using MPS device")
else:
    device = torch.device("cpu") # Defaults to CPU
    print("MPS device not found, using CPU")

# Example: Move a tensor or model to the MPS device
class TwoBranch(nn.Module):
    def __init__(self):
        super().__init__()
        model = nn.LSTM(input_size=3, hidden_size=4, batch_first=False, num_layers=1).to(device)
        self.left = model
        self.right = model
        self.combine = nn.Linear(8, 4).to(device)

    def forward(self, x_left, x_right):
        _, (l_h, _) = self.left(x_left)
        _, (r_h, _) = self.right(x_right)

        l = l_h[-1]  # last layer, shape: (batch, hidden_size)
        r = r_h[-1]

        cat = torch.cat([l, r], dim=-1)  # shape: (batch, 10)
        return self.combine(cat)          # shape: (batch, 10) -> Linear(10,1) -> (batch,1)
class BigModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.branch = TwoBranch()
        self.rest = nn.Sequential(
            # nn.ReLU(),
            # nn.Linear(4, 4),
            # nn.ReLU(),
            # nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        ).to(device)

    def forward(self, x_left, x_right):
        x = self.branch(x_left, x_right)
        return self.rest(x)
model2=BigModel().to(device)



MPS device not found, using CPU


In [40]:
"""
batch = 32
seq_len = 64

x_left  = torch.randn(seq_len, batch, 3, device=device)
x_right = torch.randn(seq_len, batch, 3, device=device)

# labels depend on problem:
y = torch.randn(batch, 1, device=device)  # regression
"""

'\nbatch = 32\nseq_len = 64\n\nx_left  = torch.randn(seq_len, batch, 3, device=device)\nx_right = torch.randn(seq_len, batch, 3, device=device)\n\n# labels depend on problem:\ny = torch.randn(batch, 1, device=device)  # regression\n'

In [41]:
import pandas as pd
import glob
files = glob.glob("data/*.json")
people = []
for file in files:
    df = pd.read_json(file)
    drawings = []
    for drawing in df.values:
        points = drawing[0][0]["points"]
        triples = [(point["x"], point["y"], point["time"]) for point in points]
        drawings.append(triples)
    people.append(drawings)


data = people


In [42]:
import json
from pathlib import Path

out_path = Path("output/data.json")

out_path.write_text(json.dumps(data, indent=2))

485185

In [43]:
out_path = Path("output/data.json")
loaded = json.load(out_path.open())

Looking at your code, I can see several issues with the train/test split creation:

1. You're using `len(left)` but `left` is not defined - it should be `len(paired)`
2. You're overwriting `leftTrain`, `rightTrain`, and `yTrain` instead of creating separate test variables
3. The variable naming is inconsistent

Here's the fixed code for your cell:



In [44]:
import numpy as np

training: list[list[list[tuple[int, int, int]]]] = loaded
pairedDeep = [[torch.tensor(drawing, device=device) for drawing in drawings] for drawings in training]
paired: list[torch.Tensor] = []
personIDs: list[int] = []
for personID, person in enumerate(pairedDeep):
    person = [drawing[:-1]-drawing[1:] for drawing in person]
    a = [personID] * len(person)
    personIDs.extend(a)
    paired.extend([((i[:, :] - i.min(dim=0, keepdim=True).values) / (
    (i.max(dim=0, keepdim=True).values - i.min(dim=0, keepdim=True).values)))*2-0.5 for i in person])
personIDs = np.array(personIDs)
lengths = torch.tensor([len(s) for s in paired])


def packPad(pairs: list[torch.Tensor], lengths: torch.Tensor, indeces):
    padded = nn.utils.rnn.pad_sequence([pairs[i].to(device) for i in indeces], batch_first=True).to(device)
    lengths = lengths[indeces]
    packed = nn.utils.rnn.pack_padded_sequence(
        padded,
        lengths,
        batch_first=True,
        enforce_sorted=False
    ).float().to(device)
    return packed



# Split at PERSON level to prevent any leakage
n_people = len(pairedDeep)
split_person_idx = int(n_people * 0.8)

# Get all indices for train people and test people
train_person_indices = []
test_person_indices = []

for person_id in range(n_people):
    person_mask = personIDs == person_id
    person_samples = np.where(person_mask)[0]

    if person_id < split_person_idx:
        train_person_indices.extend(person_samples)
    else:
        test_person_indices.extend(person_samples)

train_person_indices = np.array(train_person_indices)
test_person_indices = np.array(test_person_indices)

print(f"Training people: 0-{split_person_idx-1}")
print(f"Test people: {split_person_idx}-{n_people-1}")
print(f"Train samples: {len(train_person_indices)}")
print(f"Test samples: {len(test_person_indices)}")


def generate_balanced_pairs(indices, n_pairs, personIDs):
    """Generate 50/50 same-person vs different-person pairs"""
    left_indices = []
    right_indices = []

    n_same = n_pairs // 2
    n_diff = n_pairs - n_same

    # Same-person pairs
    for _ in range(n_same):
        idx = np.random.choice(indices)
        person_id = personIDs[idx]
        same_person_mask = personIDs[indices] == person_id
        same_person_indices = indices[same_person_mask]

        if len(same_person_indices) > 1:
            pair = np.random.choice(same_person_indices, size=2, replace=False)
            left_indices.append(pair[0])
            right_indices.append(pair[1])
        else:
            left_indices.append(idx)
            right_indices.append(idx)

    # Different-person pairs
    for _ in range(n_diff):
        idx1 = np.random.choice(indices)
        person1 = personIDs[idx1]
        diff_person_mask = personIDs[indices] != person1
        diff_person_indices = indices[diff_person_mask]

        if len(diff_person_indices) > 0:
            idx2 = np.random.choice(diff_person_indices)
            left_indices.append(idx1)
            right_indices.append(idx2)
        else:
            left_indices.append(idx1)
            right_indices.append(np.random.choice(indices))

    return np.array(left_indices), np.array(right_indices)


# Generate balanced pairs
n_train_pairs = len(train_person_indices) * 3  # Reduce multiplier for stability
n_test_pairs = len(test_person_indices) * 3

pLT, pRT = generate_balanced_pairs(train_person_indices, n_train_pairs, personIDs)
pLTest, pRTest = generate_balanced_pairs(test_person_indices, n_test_pairs, personIDs)

pairedNP = paired
lengthsNP = lengths
leftTrain = packPad(pairedNP, lengthsNP, pLT).to(device)
rightTrain = packPad(pairedNP, lengthsNP, pRT).to(device)
yTrain = (torch.tensor(personIDs[pLT] == personIDs[pRT])
          .reshape((-1, 1))
          .to(device, dtype=torch.float32))

leftTest = packPad(pairedNP, lengthsNP, pLTest).to(device)
rightTest = packPad(pairedNP, lengthsNP, pRTest).to(device)
yTest = (torch.tensor(personIDs[pLTest] == personIDs[pRTest])
         .reshape((-1, 1))
         .to(device, dtype=torch.float32))

print(f"\nTrain: {(yTrain == 1).sum().item() / len(yTrain) * 100:.1f}% same person")
print(f"Test:  {(yTest == 1).sum().item() / len(yTest) * 100:.1f}% same person")


Training people: 0-7
Test people: 8-9
Train samples: 134
Test samples: 33

Train: 50.0% same person
Test:  49.5% same person


In [45]:
str(((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean().item())

'0.5050504803657532'

In [46]:
model2.branch.left.all_weights

[[Parameter containing:
  tensor([[ 0.1634,  0.2514,  0.3924],
          [-0.4762, -0.0926,  0.4312],
          [-0.3831,  0.4893, -0.3971],
          [ 0.2921,  0.4923, -0.2561],
          [-0.1813, -0.3747, -0.2108],
          [-0.1567, -0.4283,  0.2375],
          [ 0.3409,  0.2913,  0.3565],
          [ 0.3852,  0.1670, -0.1862],
          [-0.3056, -0.2157, -0.4547],
          [ 0.0259, -0.4679,  0.4669],
          [-0.1497, -0.1768,  0.1819],
          [ 0.0415,  0.4443, -0.1662],
          [-0.1289, -0.2218, -0.4463],
          [ 0.0703, -0.1757, -0.1563],
          [ 0.0690,  0.0286,  0.4764],
          [ 0.2091,  0.4792,  0.1672]], requires_grad=True),
  Parameter containing:
  tensor([[-0.1333,  0.0458,  0.0602, -0.3342],
          [-0.3707, -0.0972,  0.3718, -0.1523],
          [-0.1391, -0.1258, -0.1378, -0.1478],
          [-0.3191, -0.1683,  0.4646,  0.3791],
          [-0.2949,  0.2273,  0.1897,  0.3721],
          [ 0.1991, -0.2522, -0.3691, -0.0788],
          [-0.2792

In [53]:
epochs = 1000
# BCELogits with L2 reg
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = torch.optim.AdamW(model2.parameters(), lr=0.1, weight_decay=1e-1)


In [60]:
annealer = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
for epoch in range(epochs):
    optimizer.zero_grad()
    epoch_loss = 0.0
    output = model2(leftTrain, rightTrain)
    loss = criterion(output, yTrain)

    loss.backward()
    optimizer.step()
    annealer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item() * yTrain.size(0)

    # average loss for the epoch
    epoch_loss /= len(leftTrain)



    if epoch % 10 == 0:
        print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

model2

Epoch   0 – loss: 22.6628 - fPC 0.7576, 0.9502
Epoch  10 – loss: 44.6237 - fPC 0.5354, 0.8134
Epoch  20 – loss: 44.5145 - fPC 0.5657, 0.8458
Epoch  30 – loss: 40.0377 - fPC 0.7374, 0.8557
Epoch  40 – loss: 37.5079 - fPC 0.7980, 0.8706
Epoch  50 – loss: 36.3260 - fPC 0.8182, 0.8682
Epoch  60 – loss: 36.8000 - fPC 0.8788, 0.8682
Epoch  70 – loss: 35.2756 - fPC 0.7980, 0.8756
Epoch  80 – loss: 33.9999 - fPC 0.8081, 0.8881
Epoch  90 – loss: 33.0253 - fPC 0.7980, 0.8955
Epoch 100 – loss: 35.5642 - fPC 0.8586, 0.8731
Epoch 110 – loss: 34.0721 - fPC 0.8687, 0.8806
Epoch 120 – loss: 33.0232 - fPC 0.8788, 0.8781
Epoch 130 – loss: 31.6479 - fPC 0.8788, 0.8955
Epoch 140 – loss: 30.5784 - fPC 0.9293, 0.9005
Epoch 150 – loss: 29.1497 - fPC 0.9394, 0.9154


KeyboardInterrupt: 

In [98]:
for epoch in range(5):
    optimizer.zero_grad()
    epoch_loss = 0.0
    output = model2(leftTrain, rightTrain)
    loss = criterion(output, yTrain)

    loss.backward()
    optimizer.step()
    annealer.step()
    optimizer.zero_grad()

    epoch_loss += loss.item() * yTrain.size(0)

    # average loss for the epoch
    epoch_loss /= len(leftTrain)



print(f"Epoch {epoch:3d} – loss: {epoch_loss:.4f} - fPC {((model2(leftTest, rightTest)>0)==(yTest>0.5)).float().mean():.4f}, {((model2(leftTrain, rightTrain)>0)==(yTrain>0.5)).float().mean():.4f}")

Epoch   4 – loss: 69.5303 - fPC 0.4848, 0.4602


In [59]:
model2 = model

In [87]:
model = model2

In [70]:
ckpt = {
    'epoch'         : epoch,                     # current epoch number
    'model_state'   : model.state_dict(),        # model parameters
    'optimizer_state': optimizer.state_dict(),   # optimizer internals
    'scheduler_state': annealer.state_dict() if annealer else None,
}
torch.save(ckpt, 'model.pt')
torch.save(model.state_dict(), 'model_weights.pt')


In [99]:
import torch

def confusion_rates(yhat: torch.Tensor, y: torch.Tensor):
    tp = (yhat == 1) & (y == 1)
    tn = (yhat == 0) & (y == 0)
    fp = (yhat == 1) & (y == 0)
    fn = (yhat == 0) & (y == 1)

    TP = int(tp.sum().item())
    TN = int(tn.sum().item())
    FP = int(fp.sum().item())
    FN = int(fn.sum().item())

    eps = 1e-12
    TPR = TP / (TP + FN + eps)   # recall / sensitivity
    TNR = TN / (TN + FP + eps)   # specificity
    FPR = FP / (FP + TN + eps)   # 1 - specificity
    FNR = FN / (FN + TP + eps)   # 1 - recall

    return {"TP": TP, "TN": TN, "FP": FP, "FN": FN,
            "TPR": TPR, "TNR": TNR, "FPR": FPR, "FNR": FNR}

def metrics(model, leftTest, rightTest, yTest):
    yhat = model(leftTest, rightTest) > 0
    y = yTest
    return confusion_rates(yhat, y)

res = metrics(model2, leftTest, rightTest, yTest)

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")


True Positive Rate (Recall) : 0.0204; TP : 1
True Negative Rate (Spec.)  : 0.9400; TN : 47
False Positive Rate         : 0.0600; FP : 3
False Negative Rate         : 0.9796; FN : 48


In [100]:
weights = torch.load('model_weights.pt', map_location=device)
model_new = BigModel()          # the same class you used during training
model_new.load_state_dict(weights)
model_new.to(device)

res = metrics(model_new, leftTest, rightTest, yTest)

print(f"True Positive Rate (Recall) : {res['TPR']:.4f}; TP : {res['TP']}")
print(f"True Negative Rate (Spec.)  : {res['TNR']:.4f}; TN : {res['TN']}")
print(f"False Positive Rate         : {res['FPR']:.4f}; FP : {res['FP']}")
print(f"False Negative Rate         : {res['FNR']:.4f}; FN : {res['FN']}")

True Positive Rate (Recall) : 0.8163; TP : 40
True Negative Rate (Spec.)  : 0.9800; TN : 49
False Positive Rate         : 0.0200; FP : 1
False Negative Rate         : 0.1837; FN : 9
